# Import packages

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import asyncio
import scipy.io as sio
from scipy.io import loadmat  # Import loadmat
# import matlab.engine
from matplotlib.colors import ListedColormap
import time
import os
import multiprocessing as mp
from scipy.interpolate import griddata
from scipy.linalg import cholesky
import sampyl as smp
import sys
import multiprocessing as mp
from numba import jit
from matplotlib.colors import ListedColormap
# Set print options to increase precision
np.set_printoptions(precision=20, suppress=False, formatter={'complexfloat': '{: 0.4f}'.format})
from concurrent.futures import ProcessPoolExecutor


In [4]:
def background(internal_loop):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, internal_loop, *args, **kwargs)
    return wrapped

class AII_model:
    def __init__(self, eta):
        # This code specifically learns eta by fixing all other parameters
        mu_phi, mu_theta, kappa = 1 , 1.57 , 0.15
        self.condRef = 8.7e5
        self.sigxx = 8.97e5
        self.sigzz = 8.43e5
        self.coil_rad_x = 2.1509e6
        self.coil_rad_y = 2.1509e6
        self.mu_phi = mu_phi
        self.mu_theta = mu_theta
        self.eta = eta
        self.kappa = kappa
        self.data_to_append = []
        self.phi_theta_sigma_data = {}
        self.V1 , self.V2, self.V3 , self.lambda1, self.lambda2, self.lambda3  = self.compute_bingham_distributions()
        self.pdf_value1 = self.calculate_mx_value()
        self.candidates = []
        self.eulerangles = []
        self.microstructure_list = {}
        self.number_of_microstructures = 30
        #self.number_of_microstructures = 1

    async def async_init(self):
        await self.run_simulation()

    # Defining the binghm distribution
    def compute_bingham_distributions(self):
        V1 = np.array([
                np.cos(self.mu_theta/2)*np.cos(self.mu_phi/2),
                -1*np.sin(self.mu_theta/2)*np.cos(self.mu_phi/2),
                -1*np.sin(self.mu_theta/2)*np.sin(self.mu_phi/2),
                -1*np.cos(self.mu_theta/2)*np.sin(self.mu_phi/2)
        ])
        V2 = np.array([
            -0.5 * np.cos(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            0.5 * np.sin(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            -0.5 * np.sin(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.cos(self.mu_phi / 2)
        ])
        V3 = np.array([
            -0.5 * np.sin(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            0.5 * np.sin(self.mu_theta/2) * np.sin(self.mu_phi / 2)
        ])
        l1 = -0.5
        l2 = l1 / self.eta
        l3 = l2 / self.kappa
        
        return V1 , V2 , V3 , l1 , l2 , l3    #, V21 , V22 , V23

    # Computing pdf value
    def pi1_x(self, x):
        term1 = self.lambda1 * (np.dot(self.V1.T, x)) ** 2
        term2 = self.lambda2 * (np.dot(self.V2.T, x)) ** 2
        term3 = self.lambda3 * (np.dot(self.V3.T, x)) ** 2

        sum_terms = term1 + term2 + term3

        pi_x_value = np.exp(sum_terms)
        return pi_x_value
    
    # This is the max pdf value, it turns out that the maximum pdf value turns out to be 1 only
    def calculate_mx_value(self):
        return 1
            
    # Accepting or rejecting quarternions
    def rununtiltrue(self):
        while True:
            x_candidate = self.generaterandomquaternion()
            f_candidate = self.pi1_x(x_candidate)
            alpha = f_candidate / self.pdf_value1
            if alpha >= 0.98:
                return x_candidate

    # Generating quarternions, this uses the method described in the thesis
    def generaterandomquaternion(self):
        # Sample three uniform random variables
        u1, u2, u3 = np.random.rand(3)

        # Compute the quaternion components
        q = np.array([
            np.sqrt(1-u1) * np.sin(2*np.pi*u2),
            np.sqrt(1-u1) * np.cos(2*np.pi*u2),
            np.sqrt(u1) * np.sin(2*np.pi*u3),
            np.sqrt(u1) * np.cos(2*np.pi*u3)
        ])
        return q
    
    @staticmethod
    def quaternion_to_rotation_matrix(q):
        # Compute the associated rotation matrix from the quaternion
        R = np.array([
            [np.square(q[0]) + np.square(q[1]) - np.square(q[2]) - np.square(q[3]), 2*(q[1]*q[2] + q[0]*q[3]), 2*(q[1]*q[3] - q[0]*q[2])],
            [2*(q[1]*q[2] - q[0]*q[3]), np.square(q[0]) - np.square(q[1]) + np.square(q[2]) - np.square(q[3]), 2*(q[3]*q[2] + q[0]*q[1])],
            [2*(q[3]*q[1] + q[0]*q[2]), 2*(q[3]*q[2] - q[0]*q[1]), np.square(q[0]) - np.square(q[1]) - np.square(q[2]) + np.square(q[3])]
        ])
        return R
    
    @staticmethod
    def rotation_matrix_to_euler_angles(R):
        second_angle = np.arccos(R[2][2])
        if second_angle > 0:
            first_angle = np.arctan2(R[2][0], -R[2][1])
            third_angle = np.arctan2(R[0][2], R[1][2])
        elif second_angle == 0:
            first_angle = np.arctan2(R[0][1], R[0][0])
            third_angle = 0
        return first_angle, second_angle, third_angle

    @staticmethod
    def quaternion_angular_difference(q1, q2):
        # Ensure the quaternions are normalized
        q1 = q1 / np.linalg.norm(q1)
        q2 = q2 / np.linalg.norm(q2)
        # Calculate the dot product between the two quaternions
        dot_product = np.dot(q1, q2)
        # Clamp the dot product to avoid numerical issues
        dot_product = np.clip(dot_product, -1.0, 1.0)
        # Calculate the angular difference in radians
        angle = 2 * np.arccos(dot_product)
        # Convert radians to degrees
        return angle
    
    async def process_element(self, i, j, data_list):
        x_candidate1 = self.rununtiltrue()
        self.candidates.append(x_candidate1)
        angle1 = self.rotation_matrix_to_euler_angles(self.quaternion_to_rotation_matrix(x_candidate1))
        first1, second1, third1 = angle1
        self.eulerangles.append(angle1)
        data_list.append((i, j, first1, second1, third1))
            
    # Creating 25X25 microtexture
    async def process_microstructure(self, val):
        """Process each microstructure in parallel."""
        data_list = []
        tasks = []
        for i in range(25):
            for j in range(25):
                tasks.append(self.process_element(i, j, data_list))

        await asyncio.gather(*tasks)
        data_list.sort(key=lambda x: (x[0], x[1]))
        self.microstructure_list[val] = data_list
        # print(f"Printing for {val} microstructure")
    
    async def run_simulation(self):
        """Parallelize the outer loop using asyncio tasks."""
        start_time = time.time()
        outer_tasks = [self.process_microstructure(val) for val in range(1, self.number_of_microstructures+1)]
        await asyncio.gather(*outer_tasks)
        end_time = time.time() - start_time
        print(f"Time taken by run_simulation: {end_time} seconds")
    
    def write_data_to_new_file(self,file_path):
        # Check if the file exists, delete it if it does
        if os.path.exists(file_path):
            os.remove(file_path)
        # Open the file in append mode ('a') or create it if it doesn't exist ('a+')
        with open(file_path, 'a') as file:
            # Append each line of data to the file
            for line in self.microstructure_list[1]:
                file.write(' '.join(map(str, line)) + '\n')  # Add a newline character to separate entries
                    
    # Running the ect Scan process
    def scan_operation(self): 
        microstructure = self.microstructure_list[1]
        # Extracting X, Y, phi, and theta values from each microstructure
        # Scaling the X value from 0 to 25 to 8 to 6176
        # This is done so that we match the range to that of the original microtexture data
        X = np.array([tup[0] for tup in microstructure]) * (6176/24) + 8
        # Reshaping the list to (25,25)
        X = X.reshape((25, 25)).T
        # Stacking the same (x,x) layer together to make a 3d microtexture of 2 layers
        X = np.stack((X, X), axis=2)

        # Scaling the Y value from 0 to 25 to 8 6312
        Y = np.array([tup[1] for tup in microstructure]) * (6312/24) + 8
        Y = Y.reshape((25, 25)).T
        Y = np.stack((Y, Y), axis=2)

        self.X = X
        self.Y = Y
        
        # Looping through microtexture list
        for val, microstructure in self.microstructure_list.items():

            # Getting the phi euler angle
            phi = np.array([tup[2] for tup in microstructure])
            phi = phi.reshape((25, 25)).T
            phi = np.stack((phi, phi), axis=2)
            cphi = np.cos(phi)
            sphi = np.sin(phi)

            # Getting the theta euler angle
            theta = np.array([tup[3] for tup in microstructure])
            theta = theta.reshape((25, 25)).T
            theta = np.stack((theta, theta), axis=2)
            stheta = np.sin(theta)

            # Storing the result in microstructure_list dictionary
            self.microstructure_list[val] = {
                'cphi': cphi,
                'sphi': sphi,
                'stheta': stheta,
                'microstructure': microstructure
            }
        
        # Scandims to set the scanning parameters
        # scanStart defines the starting location of microtexture
        # scanLength defines the total movement that will happen across all our the scans
        # scanStep defines the movement of microtexture between consecutive iteration
        scanDims = {'scanStart': np.array([0.5, 0.5, 0.5]), 'scanLength': np.array([5, 5, 5]), 'scanStep': np.array([0.2, 0.2, 0.2])}

        # The above values are in mm, so mulitplied by 1000
        scanDims['scanStart'] *= 1000
        scanDims['scanLength'] *= 1000
        scanDims['scanStep'] *= 1000

        # Calculate number of scans in each direction
        numXPoints = int(scanDims['scanLength'][0] / scanDims['scanStep'][0]) + 1
        numYPoints = int(scanDims['scanLength'][1] / scanDims['scanStep'][1]) + 1

        # Set up the scan structure
        scan = np.zeros((numXPoints, numYPoints), dtype=object)
        for ixpos in range(numXPoints):
            for iypos in range(numYPoints):
                scan[ixpos, iypos] = {
                    'num': 0,
                    'pos': [
                        scanDims['scanStart'][0] + scanDims['scanStep'][0] * ixpos,
                        scanDims['scanStart'][1] + scanDims['scanStep'][1] * iypos
                    ],
                    'area': [],
                    'total': 0
                }

        # We have already cached the electric field values and using that    
        Exs = np.load('Exs_25_26_iterations.npy')
        Eys = np.load('Eys_25_26_iterations.npy')

        # To store the mul as described in the Homa paper
        mul = np.zeros(scan.shape, dtype=np.complex128)

        for val, microstructure_data in self.microstructure_list.items():
            # Initialize scanPoint
            scanPoint = 0

            # Initialize ectScan with the same structure as scan but with all elements set to zero
            ectScan = {
                'val': val,
                'XD': np.zeros(scan.shape, dtype=float),
                'YD': np.zeros(scan.shape, dtype=float),
                'zl': np.zeros(scan.shape, dtype=np.complex128),
                'Xsub_slice': np.empty(scan.shape, dtype=object),  
                'Ysub_slice': np.empty(scan.shape, dtype=object),  
                'Zsub_slice': np.empty(scan.shape, dtype=object)
            }
        
            sphi_sub = microstructure_data['sphi']
            stheta_sub = microstructure_data['stheta']
            cphi_sub = microstructure_data['cphi']

            # Compute sigma values once per microstructure
            sigma_11 = self.sigxx + (self.sigzz - self.sigxx) * sphi_sub ** 2 * stheta_sub ** 2
            sigma_12 = ((self.sigxx - self.sigzz) * (sphi_sub * cphi_sub * (stheta_sub ** 2))) / 2
            sigma_22 = self.sigxx + ((self.sigzz - self.sigxx) * (stheta_sub ** 2)) + ((self.sigxx - self.sigzz) * (sphi_sub ** 2) * (stheta_sub ** 2))

            numX, numY = scan.shape
            real_partf_list = np.empty(scan.shape, dtype=object)
            imag_partf_list = np.empty(scan.shape, dtype=object)
            for ix in range(numX):
                for iy in range(numY):

                    X = self.X
                    Y = self.Y
                    x = np.squeeze(X[0, :, 0])
                    y = np.squeeze(Y[:, 0, 0])

                    # Min and max distances from coil to position in x and y directions
                    cx_lim_low = scan[ix, iy]['pos'][0] - self.coil_rad_x
                    cx_lim_hi = scan[ix, iy]['pos'][0] + self.coil_rad_x
                    cy_lim_low = scan[ix, iy]['pos'][1] - self.coil_rad_y
                    cy_lim_hi = scan[ix, iy]['pos'][1] + self.coil_rad_y

                    # Find the indices for x and y using numpy
                    lowIndX = np.argmax(x >= cx_lim_low)
                    highIndX = np.argmax(x >= cx_lim_hi) - 1 if np.any(x >= cx_lim_hi) else len(x) - 1
                    lowIndY = np.argmax(y >= cy_lim_low)
                    highIndY = np.argmax(y >= cy_lim_hi) - 1 if np.any(y >= cy_lim_hi) else len(y) - 1
                    lowIndZ = 0
                    highIndZ = 1

                    # Extract subarrays for X, Y, Z

                    Xsub = X[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][0]

                    Ysub = Y[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][1]

                    array1 = np.ones((X.shape[0], X.shape[1], 1)) * 8
                    array2 = np.ones((X.shape[0], X.shape[1], 1)) * 72

                    # Stack these arrays along the third dimension
                    Zsub = np.concatenate((array1, array2), axis=2)
                    Ex = Exs[scanPoint]   
                    Ey = Eys[scanPoint]

                    # Components of the approximate impedance integral (AII)
                    f1 = Ex ** 2 * (self.condRef - sigma_11)
                    f2 = -2 * Ex * Ey * sigma_12
                    f3 = Ey ** 2 * (self.condRef - sigma_22)
                    f = f1 + f2 + f3
                    real_partf_list[ix, iy] = np.real(f)
                    imag_partf_list[ix, iy] = np.imag(f)
                    
                    Xsub_slice = (np.squeeze(Xsub[0, :, 0]) / 1e6)
                    Ysub_slice = (np.squeeze(Ysub[:, 0, 0]) / 1e6)
                    Zsub_slice = (np.squeeze(Zsub[0, 0, :]) / 1e6)
                    
                    integral_inner = np.trapz(f, x=Xsub_slice, axis=1).reshape(25,1,2)
                    integral_middle = np.trapz(integral_inner, x=Ysub_slice, axis=0).reshape(1,1,2)
                    zl = np.trapz(integral_middle, x=Zsub_slice, axis=2)

                    ectScan['XD'][ix, iy] = scan[ix, iy]['pos'][0] / 1000  # Back to mm
                    ectScan['YD'][ix, iy] = scan[ix, iy]['pos'][1] / 1000  # Back to mm
                    ectScan['Xsub_slice'][ix, iy] = Xsub_slice
                    ectScan['Ysub_slice'][ix, iy] = Ysub_slice
                    ectScan['Zsub_slice'][ix, iy] = Zsub_slice
                    ectScan['zl'][ix, iy] = zl
                    
                    mul[ix, iy] += ectScan['zl'][ix, iy]
                    
                    scanPoint += 1
            microstructure_data['ectScan'] = ectScan
            microstructure_data['realpartlist'] = real_partf_list
            microstructure_data['imagpartlist'] = imag_partf_list

        mul = mul/self.number_of_microstructures

        # Store the 'mul' result back in self if needed
        self.mul = mul

    # Calculating the forward model parameters 
    def calculate_covariance_matrix(self):
        num_microstructures = len(self.microstructure_list)
        num_elements = 676  # 26 * 26
        single_side = 26 # This is the number of scans in each direction and not the microtexture length
        # Initialize covariance matrices
        # This is gamma as defined in the paper
        gamma = np.zeros((num_elements, num_elements), dtype=np.complex128)
        # This is gamma hat as defined in the paper
        gamma_hat = np.zeros((num_elements, num_elements), dtype=float)
        
        # Ect Scan will be of dimensions (26X26)
        # Gamma and gamma_hat will be of dimensions (676X676) i.e.(26X26) X (26X26)
        for l in range(num_elements):
            for k in range(l + 1):
                sum_zl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    ectScan = microstructure_data['ectScan']
                    zl_flatten = ectScan['zl'].flatten()
                    # Get zl for l and k
                    zl_l = zl_flatten[l]
                    zl_k = zl_flatten[k]

                    sum_zl_product += zl_l * zl_k 
                    
                # gamma is defined as per the definition of gamma in the paper
                gamma[l, k] = sum_zl_product / self.number_of_microstructures  - (self.mul.flat[l]*self.mul.flat[k])
                # gamma is covariance matrix, so upper triangular and lower triangular values will be same
                gamma[k, l] = gamma[l, k]
            
        self.gamma = gamma
        
        # Calculating gamma_hat as per the definiton in the paper
        for l in range(num_elements):  # num_elements is the size of the matrix
            for k in range(l + 1):
                sum_zlk_product = 0
                sum_zkl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    i, j = divmod(l, single_side)
                    m, n = divmod(k, single_side)

                    # Getting the real part of zl
                    realpartl = microstructure_data['realpartlist'][i,j]
                    # Getting the imaginary part of zl
                    imagpartl = microstructure_data['imagpartlist'][i,j]
                    
                    # Getting the real part of zk
                    realpartk = microstructure_data['realpartlist'][m,n]
                    # Getting the imaginary part of zk
                    imagpartk = microstructure_data['imagpartlist'][m,n]
                    
                    # Getting the x,y,z for doing the integration
                    Xsub_slicel = microstructure_data['ectScan']['Xsub_slice'][i,j]
                    Ysub_slicel = microstructure_data['ectScan']['Ysub_slice'][i,j]
                    Zsub_slicel = microstructure_data['ectScan']['Zsub_slice'][i,j]
                    
                    Xsub_slicek = microstructure_data['ectScan']['Xsub_slice'][m,n]
                    Ysub_slicek = microstructure_data['ectScan']['Ysub_slice'][m,n]
                    Zsub_slicek = microstructure_data['ectScan']['Zsub_slice'][m,n]
                    
                    # Performing the integration
                    integral_inner_l_lk = np.trapz(realpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_lk = np.trapz(integral_inner_l_lk, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zlk_l = np.trapz(integral_middle_l_lk, x=Zsub_slicel, axis=2)

                    integral_inner_k_lk = np.trapz(imagpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_lk = np.trapz(integral_inner_k_lk, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zlk_k = np.trapz(integral_middle_k_lk, x=Zsub_slicek, axis=2)

                    integral_inner_k_kl = np.trapz(realpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_kl = np.trapz(integral_inner_k_kl, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zkl_k = np.trapz(integral_middle_k_kl, x=Zsub_slicek, axis=2)

                    integral_inner_l_kl = np.trapz(imagpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_kl = np.trapz(integral_inner_l_kl, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zkl_l = np.trapz(integral_middle_l_kl, x=Zsub_slicel, axis=2)
                    
                    # Compute product and accumulate
                    sum_zlk_product += zlk_l * zlk_k
                    sum_zkl_product += zkl_l * zkl_k

                # Calculate the average across all microstructures
                gamma_hat[l, k] = sum_zlk_product / self.number_of_microstructures  - (np.real(self.mul.flat[l])*np.imag(self.mul.flat[k]))
                gamma_hat[k, l] = sum_zkl_product / self.number_of_microstructures  - (np.imag(self.mul.flat[l])*np.real(self.mul.flat[k]))

        self.gamma_hat = gamma_hat

In [7]:
def abc_likelihood(z_re, z_im, mu_re, mu_im, Gamma_re, Gamma_im, delta_re, delta_im, Gamma_hat , prnt = False):
    """
    Perform Approximate Bayesian Computation (ABC) likelihood estimation.
    
    Parameters:
    z_re (np.array): Real part of the observed EC measurements.
    z_im (np.array): Imaginary part of the observed EC measurements.
    mu_re (np.array): Mean of the real EC response for a fixed ODF.
    mu_im (np.array): Mean of the imaginary EC response for a fixed ODF.
    Gamma_re (np.array): Covariance matrix of the real EC response for a fixed ODF. ( It is intially gamme_re_bar )
    Gamma_im (np.array): Covariance matrix of the imaginary EC response for a fixed ODF. ( It is intially gamme_im_bar )
    delta_re (float): Standard deviation of measurement noise for real part.
    delta_im (float): Standard deviation of measurement noise for imaginary part.
    Gamma_hat (np.array): This is the covariance_bar calculated above.
    
    Returns:
    float: ABC likelihood value.
    """
    
    # Equation 21: Update Gamma matrices with noise
    Gamma_re += delta_re ** 2 * np.eye(Gamma_re.shape[0])
    Gamma_im += delta_im ** 2 * np.eye(Gamma_im.shape[0])

    # Concatenate real and imaginary parts
    z = np.concatenate((z_re, z_im))
    mu = np.concatenate((mu_re.flatten(), mu_im.flatten()))
    # Create the combined covariance matrix Lambda
    if Gamma_hat is None:
        Gamma_hat = np.zeros_like(Gamma_re)
    # There is no imaginary part in Gamma_re, gamma_im and gamma_hat
    Lambda = np.block([
        [Gamma_re, Gamma_hat],
        [Gamma_hat.T, Gamma_im]
    ])
    Lambda = make_positive_definite(Lambda)

    # Cholesky factorization
    R = cholesky(Lambda, lower=True)

    # Compute the standard normal random vector w
    w = np.linalg.solve(R.T, z - mu)
    
    # Equation 22: Summary statistics
    w_norm = np.linalg.norm(w) 
    w_mean = np.mean(w)
    
    # Expected values for standard normal vector
    m = z_re.shape[0]  # Number of measurements
    expected_w_norm = np.sqrt(2 * m)
    expected_w_mean = 0
    
    # Equation 23: Weighting function (Gaussian)
    sigma_norm = 1  # Assuming a standard deviation of 1 for simplicity
    weighting_function = -((2*m)*(w_mean)**2) - ((w_norm - expected_w_norm)**2 / (sigma_norm ** 2))

    if prnt:
        print(f'zeta = {z} \n')
        print(f'b = {mu}\n')
        print(f'Lambda = {Lambda}\n')
        print(f'R = {R}\n')
        print(f'w = {w} \n')
        print(f'w_norm, w_mean = {w_norm, w_mean}\n')
    return weighting_function

In [8]:
# Define the custom Metropolis sampler
class CustomMetropolis(smp.Metropolis):
    def __init__(self, *args, **kwargs):
        super(CustomMetropolis, self).__init__(*args, **kwargs)
        self._accepted = 0
        self._sampled = 0

    def proposal(self, state, scale):
        proposed = state.copy()
        for var in state:
            proposed[var] = np.random.normal(state[var], scale[var])
        return proposed

    def tune(self, scale, acceptance):
        # Handle the case when scale is a dictionary
        if isinstance(scale, dict):
            for key in scale.keys():
                if acceptance < 0.15:
                    scale[key] *= 0.1
                elif acceptance < 0.30:
                    scale[key] *= 0.5
                elif acceptance > 0.95:
                    scale[key] *= 10.0
                elif acceptance > 0.75:
                    scale[key] *= 2.0
        else:
            if acceptance < 0.15:
                scale *= 0.1
            elif acceptance < 0.30:
                scale *= 0.5
            elif acceptance > 0.95:
                scale *= 10.0
            elif acceptance > 0.75:
                scale *= 2.0
        return scale

    def step(self):
        """ Perform a Metropolis-Hastings step. """
        x = self.state
        y = self.proposal(x, scale=self.scale)
        if self._accept(x, y):
            self.state = y
            self._accepted += 1

        self._sampled += 1

        self._steps_until_tune -= 1
        if self._steps_until_tune == 0:
            self.scale = self.tune(self.scale, self.acceptance)
            self._steps_until_tune = self.tune_interval

        return self.state

    def _accept(self, x, y):
        log_accept_prob = self.model.logp(y) - self.model.logp(x)
        return np.log(np.random.uniform(0.7, 1)) < log_accept_prob

    @property
    def acceptance(self):
        return self._accepted / self._sampled if self._sampled > 0 else 0

In [10]:
import numpy as np
import asyncio
from scipy.linalg import cholesky
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

def calc_mean(posterior):
    return np.mean(posterior,0)

# Function to symmetrize a matrix
def symmetrize(matrix):
    return (matrix + matrix.T) / 2

# Function to ensure positive semi-definiteness
def make_positive_semidefinite(matrix):
    sym_matrix = symmetrize(matrix)
    eigvals, eigvecs = np.linalg.eigh(sym_matrix)
    eigvals[eigvals < 0] = 0
    return eigvecs @ np.diag(eigvals) @ eigvecs.T

def make_positive_definite(matrix, epsilon=1e-8):
    def symmetrize(mat):
        return (mat + mat.T) / 2
    
    sym_matrix = symmetrize(matrix)
    eigvals, eigvecs = np.linalg.eigh(sym_matrix)
    eigvals[eigvals < epsilon] = epsilon  # Set a small positive value to ensure all eigenvalues are positive
    return eigvecs @ np.diag(eigvals) @ eigvecs.T

async def simulate_AII_model(params):
    '''This function will simulate the microstructure given the parameters. 
    After that it will run the AII model to generate the real and imaginary predictions for the given microstructure.
    It will also compute the covariance matrices and gamma_hat
    Finally, it should return: mu_re, mu_im, Gamma_re, Gamma_im, and Gamma_hat

    The input params: contains each of the 3 parameters that define the two Bingham distributions
    '''
    # param_value = params if isinstance(params, float) else params._value  # Extract value if wrapped in ArrayBox
    param1, param2, param3, param4 = params
    aiimodel = AII_model(param3)
    await aiimodel.async_init()
    aiimodel.scan_operation()
    aiimodel.calculate_covariance_matrix()
    mu_re = np.real(aiimodel.mul)
    mu_im = np.imag(aiimodel.mul)
    gamma_re = np.real(aiimodel.gamma)
    gamma_im = np.imag(aiimodel.gamma)
    gamma_bar = aiimodel.gamma_hat
    return mu_re, mu_im, gamma_re, gamma_im, gamma_bar

def run_async(coroutine):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coroutine)

# u1(n) and u2(n) equations
# def u1(n):
#     return 2.045530 * n**2 - 1.630916 * n + 0.340652

# def u2(n):
#     return 0.88139 * n**2 - 0.496072 * n + 0.061516

# Function to represent the prior pi(η, κ)
# def eta_kappa_function(n, k):
#     if k > u1(n):
#         return np.exp(-(k - u1(n))**2)
#     elif u1(n) >= k >= u2(n):
#         return 1
#     elif k < u2(n):
#         return np.exp(-(k - u2(n))**2)
    
def log_likelihood(params):
    start_time = time.time()
#     mu_psi, mu_theta , eta, kappa  = params
    parameters = [ 1 , 1.57 , params, 0.15]
    start_time_aiimodel = time.time()
    results = run_async(simulate_AII_model(parameters))
    end_time_aiimodel = time.time()
    if results is None:
        raise ValueError("simulate_microtexture_AII returned None")
        
    mu_re, mu_im, gamma_re, gamma_im, gamma_hat = results
    delta_re = 1e-6  #2e-2          #0.05
    delta_im = 1e-6  #2e-2          #0.05
    

    likelihood = abc_likelihood(z_re, z_im, mu_re, mu_im, gamma_re, gamma_im, 
                                delta_re, delta_im, gamma_hat, prnt=False)  
    if likelihood is None:
        raise ValueError("abc_likelihood returned None")
    model = smp.Model()
    model.add(likelihood)
    # Below lines define the prior for various parameters
#     model.add(smp.uniform(mu_psi, lower=0 , upper=np.pi))
#     model.add(smp.uniform(mu_theta, lower=0 , upper=np.pi/2))
#     model.add(eta_kappa_function(eta,kappa))
    print(f"Params {params}, Likelihood {likelihood} , Model {model()}")
    return model().round(3)

### z_re and z_im as the original green subsection

In [14]:
# True value
z_re = np.loadtxt("new_green_subsection_real_ect_values.csv", delimiter=',').flatten()
z_im = np.loadtxt("new_green_subsection_imag_ect_values.csv", delimiter=',').flatten()

In [15]:
# This is the main code, run this so that inverse model will start running
# This function will take the parameter(s) for Bingham distribution and below will be the fucntion flow
# log_likelihood(parameters as input) -> calls simulate_AII_model(parameters as input) -> generates data and runs the forward model quantities
def main():
    sampler = smp.Metropolis  # Metropolis, Slice, Hamiltonian, NUTS
    num_step = 5 if sampler == smp.NUTS else 50

    Posterior_params = []
    step = 0

    start_params = 0.5  # List of initial parameters to start with
    scale = {'params': 1}
    while step < num_step:
        print(f'\n______________________________________________{step}th iteration_______________________________________')
        
        start_params_ltk = {'params': start_params}  # Use initial param or updated param
        # print(f'Params start = {start_params_ltk}')  # Print the starting model parameter
        hmc_param_ltk = CustomMetropolis(log_likelihood, start_params_ltk, scale=scale) #sampler(log_likelihood, start_params_ltk) # NUTS, Metropolis, Hamiltonian, Slice
        chain_param_ltk = hmc_param_ltk.sample(1, burn=0, n_chains=1)
        print(f'Accepted params = {chain_param_ltk}')
        # Replace current with accepted params
        start_params = calc_mean(chain_param_ltk.params)
        
        # Store the new params
        Posterior_params.extend(chain_param_ltk.params)
        
        # Repeat step 3-5 
        step += 1
        print(f'Recovered params = {start_params}')

    return Posterior_params
start_param = [ 1 , 1.57 , 0.5 , 0.15 ]
print(start_param)
# Simulate the real data
Posterior_params = main()

[1, 1.57, 0.5, 0.15]

______________________________________________0th iteration_______________________________________
Time taken by run_simulation: 11.211926937103271 seconds
Params 2.1860797405019863, Likelihood -30291022.799562804 , Model -30291022.799562804
Time taken by run_simulation: 47.58322596549988 seconds
Params 0.5, Likelihood -152338002.30026406 , Model -152338002.30026406
Progress: [##############################] 1 of 1 samples
Accepted params = [(2.1860797405019863,)]
Recovered params = 2.1860797405019863

______________________________________________1th iteration_______________________________________
Time taken by run_simulation: 9.361385107040405 seconds
Params 2.4377232388054253, Likelihood -25233816.06058162 , Model -25233816.06058162
Time taken by run_simulation: 10.385344982147217 seconds
Params 2.1860797405019863, Likelihood -31168419.69723006 , Model -31168419.69723006
Progress: [##############################] 1 of 1 samples
Accepted params = [(2.4377232388

Params 2.777640438811907, Likelihood -22917433.235001415 , Model -22917433.235001415
Progress: [##############################] 1 of 1 samples
Accepted params = [(2.777640438811907,)]
Recovered params = 2.777640438811907

______________________________________________32th iteration_______________________________________
Time taken by run_simulation: 7.385298013687134 seconds
Params 3.109607895530695, Likelihood -24202217.83345125 , Model -24202217.83345125
Time taken by run_simulation: 8.244933843612671 seconds
Params 2.777640438811907, Likelihood -21945547.89571733 , Model -21945547.89571733
Progress: [##############################] 1 of 1 samples
Accepted params = [(2.777640438811907,)]
Recovered params = 2.777640438811907

______________________________________________33th iteration_______________________________________
Time taken by run_simulation: 9.91906213760376 seconds
Params 2.2998835516641054, Likelihood -27423815.347430836 , Model -27423815.347430836
Time taken by run_simu

KeyboardInterrupt: 

In [16]:
# Calculating likelihood values for a range of parameters to verify that the likelihood value is highest for true parameter
likelihhod_values = []
for val in np.arange(-0.5, 0.5, 0.1):
    likelihhod_values.append(log_likelihood(val))

Time taken by run_simulation: 0.691457986831665 seconds
Params -0.5, Likelihood -147293612.92912063 , Model -147293612.92912063
Time taken by run_simulation: 0.7200639247894287 seconds
Params -0.4, Likelihood -140498132.06743973 , Model -140498132.06743973
Time taken by run_simulation: 0.7513449192047119 seconds
Params -0.30000000000000004, Likelihood -140385385.85283133 , Model -140385385.85283133
Time taken by run_simulation: 0.7996397018432617 seconds
Params -0.20000000000000007, Likelihood -134189678.2830719 , Model -134189678.2830719
Time taken by run_simulation: 0.679656982421875 seconds
Params -0.10000000000000009, Likelihood -129255034.6362536 , Model -129255034.6362536


/var/folders/fd/lndhxkqs20bb__svt4c6nddr0000gn/T/ipykernel_48836/963080779.py:65: RuntimeWarning: overflow encountered in exp
  pi_x_value = np.exp(sum_terms)


Time taken by run_simulation: 1.0239722728729248 seconds
Params -1.1102230246251565e-16, Likelihood -116738718.44933487 , Model -116738718.44933487
Time taken by run_simulation: 232.37617111206055 seconds
Params 0.09999999999999987, Likelihood -210243172.67846376 , Model -210243172.67846376
Time taken by run_simulation: 327.12204003334045 seconds
Params 0.19999999999999984, Likelihood -195537208.21928018 , Model -195537208.21928018
Time taken by run_simulation: 75.9569251537323 seconds
Params 0.2999999999999998, Likelihood -181467885.72988588 , Model -181467885.72988588
Time taken by run_simulation: 58.802566051483154 seconds
Params 0.3999999999999998, Likelihood -167785266.43672395 , Model -167785266.43672395


### z_re and z_im as random microtexture

In [16]:
# Run the below code to generate a random microtexture
start = time.time()
aiimodel=AII_model(0.1)
await aiimodel.async_init()
# Uncomment below line to save the data generated in a txt file which can then be used in mtex for plotting pole figures and orientation plots
# It's a good idea to change the value of self.number_of_microstructures to 1, so that it generates just 1 instance of microtexture
# aiimodel.write_data_to_new_file('recovered_parameters_green_subsection_1_5_0_0_9_0_5.txt')
aiimodel.scan_operation()
# ectScan = aiimodel.scan_operation()
# aiimodel.plot()
# aiimodel.write_data_to_new_file('test_nov_10_05.txt')
# aiimodel.calculate_covariance_matrix()

# Below line will take the first simulated microtexture and consider it as the true value 
z_re = np.real(aiimodel.microstructure_list[1]['ectScan']['zl']).flatten()
z_im = np.imag(aiimodel.microstructure_list[1]['ectScan']['zl']).flatten()

Time taken by run_simulation: 246.14950394630432 seconds


### If we want to recover all the four parameters then run the below cells

In [26]:
def background(internal_loop):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, internal_loop, *args, **kwargs)
    return wrapped

class AII_model:
    def __init__(self, mu_phi, mu_theta, eta, kappa):
#         eta, kappa = 0.25 , 0.05
#         mu_phi, mu_theta, kappa = 1 , 1.57 , 0.15
        self.condRef = 8.7e5
        self.sigxx = 8.97e5
        self.sigzz = 8.43e5
        self.coil_rad_x = 2.1509e6
        self.coil_rad_y = 2.1509e6
        self.mu_phi = mu_phi
        self.mu_theta = mu_theta
        # self.mu_theta2 = mu_theta2
        self.eta = eta
        self.kappa = kappa
        self.data_to_append = []
        self.phi_theta_sigma_data = {}
        self.V1 , self.V2, self.V3 , self.lambda1, self.lambda2, self.lambda3  = self.compute_bingham_distributions()
        self.pdf_value1 = self.calculate_mx_value()
        self.candidates = []
        self.eulerangles = []
        self.microstructure_list = {}
        self.number_of_microstructures = 30
#         self.number_of_microstructures = 1

    async def async_init(self):
        await self.run_simulation()

    def compute_bingham_distributions(self):
        V1 = np.array([
                np.cos(self.mu_theta/2)*np.cos(self.mu_phi/2),
                -1*np.sin(self.mu_theta/2)*np.cos(self.mu_phi/2),
                -1*np.sin(self.mu_theta/2)*np.sin(self.mu_phi/2),
                -1*np.cos(self.mu_theta/2)*np.sin(self.mu_phi/2)
        ])
        V2 = np.array([
            -0.5 * np.cos(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            0.5 * np.sin(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            -0.5 * np.sin(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.cos(self.mu_phi / 2)
        ])
        V3 = np.array([
            -0.5 * np.sin(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            0.5 * np.sin(self.mu_theta/2) * np.sin(self.mu_phi / 2)
        ])
        l1 = -0.5
        l2 = l1 / self.eta
        l3 = l2 / self.kappa
        
        return V1 , V2 , V3 , l1 , l2 , l3    #, V21 , V22 , V23

    def pi1_x(self, x):
        term1 = self.lambda1 * (np.dot(self.V1.T, x)) ** 2
        term2 = self.lambda2 * (np.dot(self.V2.T, x)) ** 2
        term3 = self.lambda3 * (np.dot(self.V3.T, x)) ** 2

        sum_terms = term1 + term2 + term3

        pi_x_value = np.exp(sum_terms)
        return pi_x_value
    
    def calculate_mx_value(self):
        return 1
            
    def rununtiltrue(self):
        while True:
            x_candidate = self.generaterandomquaternion()
            f_candidate = self.pi1_x(x_candidate)
            alpha = f_candidate / self.pdf_value1
            if alpha >= 0.98:
                return x_candidate

    def generaterandomquaternion(self):
        # Sample three uniform random variables
        u1, u2, u3 = np.random.rand(3)

        # Compute the quaternion components
        q = np.array([
            np.sqrt(1-u1) * np.sin(2*np.pi*u2),
            np.sqrt(1-u1) * np.cos(2*np.pi*u2),
            np.sqrt(u1) * np.sin(2*np.pi*u3),
            np.sqrt(u1) * np.cos(2*np.pi*u3)
        ])
        return q
    
    @staticmethod
    def quaternion_to_rotation_matrix(q):
        # Compute the associated rotation matrix from the quaternion
        R = np.array([
            [np.square(q[0]) + np.square(q[1]) - np.square(q[2]) - np.square(q[3]), 2*(q[1]*q[2] + q[0]*q[3]), 2*(q[1]*q[3] - q[0]*q[2])],
            [2*(q[1]*q[2] - q[0]*q[3]), np.square(q[0]) - np.square(q[1]) + np.square(q[2]) - np.square(q[3]), 2*(q[3]*q[2] + q[0]*q[1])],
            [2*(q[3]*q[1] + q[0]*q[2]), 2*(q[3]*q[2] - q[0]*q[1]), np.square(q[0]) - np.square(q[1]) - np.square(q[2]) + np.square(q[3])]
        ])
        return R
    
    @staticmethod
    def rotation_matrix_to_euler_angles(R):
        second_angle = np.arccos(R[2][2])
        if second_angle > 0:
            first_angle = np.arctan2(R[2][0], -R[2][1])
            third_angle = np.arctan2(R[0][2], R[1][2])
        elif second_angle == 0:
            first_angle = np.arctan2(R[0][1], R[0][0])
            third_angle = 0
        return first_angle, second_angle, third_angle

    @staticmethod
    def quaternion_angular_difference(q1, q2):
        # Ensure the quaternions are normalized
        q1 = q1 / np.linalg.norm(q1)
        q2 = q2 / np.linalg.norm(q2)
        # Calculate the dot product between the two quaternions
        dot_product = np.dot(q1, q2)
        # Clamp the dot product to avoid numerical issues
        dot_product = np.clip(dot_product, -1.0, 1.0)
        # Calculate the angular difference in radians
        angle = 2 * np.arccos(dot_product)
        # Convert radians to degrees
        return angle
    
    async def process_element(self, i, j, data_list):
        x_candidate1 = self.rununtiltrue()
        self.candidates.append(x_candidate1)
        angle1 = self.rotation_matrix_to_euler_angles(self.quaternion_to_rotation_matrix(x_candidate1))
        first1, second1, third1 = angle1
        self.eulerangles.append(angle1)
        data_list.append((i, j, first1, second1, third1))
            
    async def process_microstructure(self, val):
        """Process each microstructure in parallel."""
        data_list = []
        tasks = []
        for i in range(25):
            for j in range(25):
                tasks.append(self.process_element(i, j, data_list))

        await asyncio.gather(*tasks)
        data_list.sort(key=lambda x: (x[0], x[1]))
        self.microstructure_list[val] = data_list
        # print(f"Printing for {val} microstructure")
    
    async def run_simulation(self):
        """Parallelize the outer loop using asyncio tasks."""
        start_time = time.time()
        outer_tasks = [self.process_microstructure(val) for val in range(1, self.number_of_microstructures+1)]
        await asyncio.gather(*outer_tasks)
        end_time = time.time() - start_time
        print(f"Time taken by run_simulation: {end_time} seconds")

    # def write_data_to_new_file(self,file_path):
    #     # Check if the file exists, delete it if it does
    #     if os.path.exists(file_path):
    #         os.remove(file_path)
    #     # Open the file in append mode ('a') or create it if it doesn't exist ('a+')
    #     with open(file_path, 'a') as file:
    #         # Append each line of data to the file
    #         for line in self.data_to_append[:625]:
    #             file.write(' '.join(map(str, line)) + '\n')  # Add a newline character to separate entries
    
    def write_data_to_new_file(self,file_path):
        # Check if the file exists, delete it if it does
        if os.path.exists(file_path):
            os.remove(file_path)
        # Open the file in append mode ('a') or create it if it doesn't exist ('a+')
        with open(file_path, 'a') as file:
            # Append each line of data to the file
            for line in self.microstructure_list[1]:
                file.write(' '.join(map(str, line)) + '\n')  # Add a newline character to separate entries
                    
                    
    def scan_operation(self): 
        microstructure = self.microstructure_list[1]
        # Extracting X, Y, phi, and theta values from each microstructure
        X = np.array([tup[0] for tup in microstructure]) * (6176/24) + 8
        X = X.reshape((25, 25)).T
        X = np.stack((X, X), axis=2)

        Y = np.array([tup[1] for tup in microstructure]) * (6312/24) + 8
        Y = Y.reshape((25, 25)).T
        Y = np.stack((Y, Y), axis=2)

        self.X = X
        self.Y = Y
            
        for val, microstructure in self.microstructure_list.items():

            phi = np.array([tup[2] for tup in microstructure])
            phi = phi.reshape((25, 25)).T
            phi = np.stack((phi, phi), axis=2)
            cphi = np.cos(phi)
            sphi = np.sin(phi)

            theta = np.array([tup[3] for tup in microstructure])
            theta = theta.reshape((25, 25)).T
            theta = np.stack((theta, theta), axis=2)
            stheta = np.sin(theta)

            # Optionally, you can store these processed arrays in a dictionary if needed
            self.microstructure_list[val] = {
                'cphi': cphi,
                'sphi': sphi,
                'stheta': stheta,
                'microstructure': microstructure
            }
        
        scanDims = {'scanStart': np.array([0.5, 0.5, 0.5]), 'scanLength': np.array([5, 5, 5]), 'scanStep': np.array([0.2, 0.2, 0.2])}

        scanDims['scanStart'] *= 1000
        scanDims['scanLength'] *= 1000
        scanDims['scanStep'] *= 1000

        # Calculate number of points in each dimension
        numXPoints = int(scanDims['scanLength'][0] / scanDims['scanStep'][0]) + 1
        numYPoints = int(scanDims['scanLength'][1] / scanDims['scanStep'][1]) + 1

        # Set up the scan structure
        scan = np.zeros((numXPoints, numYPoints), dtype=object)
        for ixpos in range(numXPoints):
            for iypos in range(numYPoints):
                scan[ixpos, iypos] = {
                    'num': 0,
                    'pos': [
                        scanDims['scanStart'][0] + scanDims['scanStep'][0] * ixpos,
                        scanDims['scanStart'][1] + scanDims['scanStep'][1] * iypos
                    ],
                    'area': [],
                    'total': 0
                }

        Exs = np.load('Exs_25_26_iterations.npy')
        Eys = np.load('Eys_25_26_iterations.npy')

        mul = np.zeros(scan.shape, dtype=np.complex128)

        for val, microstructure_data in self.microstructure_list.items():
            # Initialize scanPoint
            scanPoint = 0

            # Initialize ectScan with the same structure as scan but with all elements set to zero
            ectScan = {
                'val': val,
                'XD': np.zeros(scan.shape, dtype=float),
                'YD': np.zeros(scan.shape, dtype=float),
                'zl': np.zeros(scan.shape, dtype=np.complex128),
                'Xsub_slice': np.empty(scan.shape, dtype=object),  
                'Ysub_slice': np.empty(scan.shape, dtype=object),  
                'Zsub_slice': np.empty(scan.shape, dtype=object)
            }
        
            sphi_sub = microstructure_data['sphi']
            stheta_sub = microstructure_data['stheta']
            cphi_sub = microstructure_data['cphi']

            # Compute sigma values once per microstructure
            sigma_11 = self.sigxx + (self.sigzz - self.sigxx) * sphi_sub ** 2 * stheta_sub ** 2
            sigma_12 = ((self.sigxx - self.sigzz) * (sphi_sub * cphi_sub * (stheta_sub ** 2))) / 2
            sigma_22 = self.sigxx + ((self.sigzz - self.sigxx) * (stheta_sub ** 2)) + ((self.sigxx - self.sigzz) * (sphi_sub ** 2) * (stheta_sub ** 2))

            numX, numY = scan.shape
            real_partf_list = np.empty(scan.shape, dtype=object)
            imag_partf_list = np.empty(scan.shape, dtype=object)
            for ix in range(numX):
                for iy in range(numY):

                    X = self.X
                    Y = self.Y
                    x = np.squeeze(X[0, :, 0])
                    y = np.squeeze(Y[:, 0, 0])

                    # Min and max distances from coil to position in x and y directions
                    cx_lim_low = scan[ix, iy]['pos'][0] - self.coil_rad_x
                    cx_lim_hi = scan[ix, iy]['pos'][0] + self.coil_rad_x
                    cy_lim_low = scan[ix, iy]['pos'][1] - self.coil_rad_y
                    cy_lim_hi = scan[ix, iy]['pos'][1] + self.coil_rad_y

                    # Find the indices for x and y using numpy
                    lowIndX = np.argmax(x >= cx_lim_low)
                    highIndX = np.argmax(x >= cx_lim_hi) - 1 if np.any(x >= cx_lim_hi) else len(x) - 1
                    lowIndY = np.argmax(y >= cy_lim_low)
                    highIndY = np.argmax(y >= cy_lim_hi) - 1 if np.any(y >= cy_lim_hi) else len(y) - 1
                    lowIndZ = 0
                    highIndZ = 1

                    # Extract subarrays for X, Y, Z

                    Xsub = X[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][0]

                    Ysub = Y[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][1]

                    array1 = np.ones((X.shape[0], X.shape[1], 1)) * 8
                    array2 = np.ones((X.shape[0], X.shape[1], 1)) * 72

                    # Stack these arrays along the third dimension
                    Zsub = np.concatenate((array1, array2), axis=2)
                    Ex = Exs[scanPoint]   
                    Ey = Eys[scanPoint]

                    # Components of the approximate impedance integral (AII)
                    f1 = Ex ** 2 * (self.condRef - sigma_11)
                    f2 = -2 * Ex * Ey * sigma_12
                    f3 = Ey ** 2 * (self.condRef - sigma_22)
                    f = f1 + f2 + f3
                    real_partf_list[ix, iy] = np.real(f)
                    imag_partf_list[ix, iy] = np.imag(f)
                    
                    Xsub_slice = (np.squeeze(Xsub[0, :, 0]) / 1e6)
                    Ysub_slice = (np.squeeze(Ysub[:, 0, 0]) / 1e6)
                    Zsub_slice = (np.squeeze(Zsub[0, 0, :]) / 1e6)
                    
                    integral_inner = np.trapz(f, x=Xsub_slice, axis=1).reshape(25,1,2)
                    integral_middle = np.trapz(integral_inner, x=Ysub_slice, axis=0).reshape(1,1,2)
                    zl = np.trapz(integral_middle, x=Zsub_slice, axis=2)

                    ectScan['XD'][ix, iy] = scan[ix, iy]['pos'][0] / 1000  # Back to mm
                    ectScan['YD'][ix, iy] = scan[ix, iy]['pos'][1] / 1000  # Back to mm
                    ectScan['Xsub_slice'][ix, iy] = Xsub_slice
                    ectScan['Ysub_slice'][ix, iy] = Ysub_slice
                    ectScan['Zsub_slice'][ix, iy] = Zsub_slice
                    ectScan['zl'][ix, iy] = zl
                    
                    mul[ix, iy] += ectScan['zl'][ix, iy]
                    
                    scanPoint += 1
            microstructure_data['ectScan'] = ectScan
            microstructure_data['realpartlist'] = real_partf_list
            microstructure_data['imagpartlist'] = imag_partf_list

        mul = mul/self.number_of_microstructures

        # Store the 'mul' result back in self if needed
        self.mul = mul

    def calculate_covariance_matrix(self):
        num_microstructures = len(self.microstructure_list)
        num_elements = 676  # 26 * 26
        single_side = 26
        # Initialize covariance matrices
        gamma = np.zeros((num_elements, num_elements), dtype=np.complex128)
        gamma_hat = np.zeros((num_elements, num_elements), dtype=float)
        
        for l in range(num_elements):
            for k in range(l + 1):
                sum_zl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    ectScan = microstructure_data['ectScan']
                    zl_flatten = ectScan['zl'].flatten()
                    # Get zl for l and k
                    zl_l = zl_flatten[l]
                    zl_k = zl_flatten[k]

                    sum_zl_product += zl_l * zl_k 
                    
                gamma[l, k] = sum_zl_product / self.number_of_microstructures  - (self.mul.flat[l]*self.mul.flat[k])
                gamma[k, l] = gamma[l, k]
            
        self.gamma = gamma
        
        for l in range(num_elements):  # num_elements is the size of the matrix
            for k in range(l + 1):
                sum_zlk_product = 0
                sum_zkl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    i, j = divmod(l, single_side)
                    m, n = divmod(k, single_side)

                    realpartl = microstructure_data['realpartlist'][i,j]
                    imagpartl = microstructure_data['imagpartlist'][i,j]
                    
                    realpartk = microstructure_data['realpartlist'][m,n]
                    imagpartk = microstructure_data['imagpartlist'][m,n]
                    
                    Xsub_slicel = microstructure_data['ectScan']['Xsub_slice'][i,j]
                    Ysub_slicel = microstructure_data['ectScan']['Ysub_slice'][i,j]
                    Zsub_slicel = microstructure_data['ectScan']['Zsub_slice'][i,j]
                    
                    Xsub_slicek = microstructure_data['ectScan']['Xsub_slice'][m,n]
                    Ysub_slicek = microstructure_data['ectScan']['Ysub_slice'][m,n]
                    Zsub_slicek = microstructure_data['ectScan']['Zsub_slice'][m,n]
                    
                    integral_inner_l_lk = np.trapz(realpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_lk = np.trapz(integral_inner_l_lk, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zlk_l = np.trapz(integral_middle_l_lk, x=Zsub_slicel, axis=2)

                    integral_inner_k_lk = np.trapz(imagpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_lk = np.trapz(integral_inner_k_lk, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zlk_k = np.trapz(integral_middle_k_lk, x=Zsub_slicek, axis=2)

                    integral_inner_k_kl = np.trapz(realpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_kl = np.trapz(integral_inner_k_kl, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zkl_k = np.trapz(integral_middle_k_kl, x=Zsub_slicek, axis=2)

                    integral_inner_l_kl = np.trapz(imagpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_kl = np.trapz(integral_inner_l_kl, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zkl_l = np.trapz(integral_middle_l_kl, x=Zsub_slicel, axis=2)
                    
                    # Compute product and accumulate
                    sum_zlk_product += zlk_l * zlk_k
                    sum_zkl_product += zkl_l * zkl_k

                # Calculate the average across all microstructures
                gamma_hat[l, k] = sum_zlk_product / self.number_of_microstructures  - (np.real(self.mul.flat[l])*np.imag(self.mul.flat[k]))
                gamma_hat[k, l] = sum_zkl_product / self.number_of_microstructures  - (np.imag(self.mul.flat[l])*np.real(self.mul.flat[k]))

        self.gamma_hat = gamma_hat

In [39]:
import numpy as np
import asyncio
from scipy.linalg import cholesky
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

def calc_mean(posterior):
    return np.mean(posterior,0)

# Function to symmetrize a matrix
def symmetrize(matrix):
    return (matrix + matrix.T) / 2

# Function to ensure positive semi-definiteness
def make_positive_semidefinite(matrix):
    sym_matrix = symmetrize(matrix)
    eigvals, eigvecs = np.linalg.eigh(sym_matrix)
    eigvals[eigvals < 0] = 0
    return eigvecs @ np.diag(eigvals) @ eigvecs.T

def make_positive_definite(matrix, epsilon=1e-8):
    def symmetrize(mat):
        return (mat + mat.T) / 2
    
    sym_matrix = symmetrize(matrix)
    eigvals, eigvecs = np.linalg.eigh(sym_matrix)
    eigvals[eigvals < epsilon] = epsilon  # Set a small positive value to ensure all eigenvalues are positive
    return eigvecs @ np.diag(eigvals) @ eigvecs.T

async def simulate_AII_model(params):
    '''This function will simulate the microstructure given the parameters. 
    After that it will run the AII model to generate the real and imaginary predictions for the given microstructure.
    It will also compute the covariance matrices and gamma_hat
    Finally, it should return: mu_re, mu_im, Gamma_re, Gamma_im, and Gamma_hat

    The input params: contains each of the 3 parameters that define the two Bingham distributions
    '''
    # param_value = params if isinstance(params, float) else params._value  # Extract value if wrapped in ArrayBox
    param1, param2, param3, param4 = params
    aiimodel = AII_model(param1, param2, param3, param4)
    await aiimodel.async_init()
    aiimodel.scan_operation()
    aiimodel.calculate_covariance_matrix()
    mu_re = np.real(aiimodel.mul)
    mu_im = np.imag(aiimodel.mul)
    gamma_re = np.real(aiimodel.gamma)
    gamma_im = np.imag(aiimodel.gamma)
    gamma_bar = aiimodel.gamma_hat
    return mu_re, mu_im, gamma_re, gamma_im, gamma_bar

def run_async(coroutine):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coroutine)

# u1(n) and u2(n) equations
# def u1(n):
#     return 2.045530 * n**2 - 1.630916 * n + 0.340652

# def u2(n):
#     return 0.88139 * n**2 - 0.496072 * n + 0.061516

# Function to represent the prior pi(η, κ)
def eta_kappa_function(n, k):
    if k > u1(n):
        return np.exp(-(k - u1(n))**2)
    elif u1(n) >= k >= u2(n):
        return 1
    elif k < u2(n):
        return np.exp(-(k - u2(n))**2)
    
def log_likelihood(params):
    start_time = time.time()
    mu_psi, mu_theta , eta, kappa  = params
    parameters = [mu_psi, mu_theta , eta, kappa]
    start_time_aiimodel = time.time()
    results = run_async(simulate_AII_model(parameters))
    end_time_aiimodel = time.time()
    if results is None:
        raise ValueError("simulate_microtexture_AII returned None")
        
    mu_re, mu_im, gamma_re, gamma_im, gamma_hat = results
    delta_re = 1e-6  #2e-2          #0.05
    delta_im = 1e-6  #2e-2          #0.05
    

    likelihood = abc_likelihood(z_re, z_im, mu_re, mu_im, gamma_re, gamma_im, 
                                delta_re, delta_im, gamma_hat, prnt=False)  
    if likelihood is None:
        raise ValueError("abc_likelihood returned None")
    model = smp.Model()
    model.add(likelihood)
#     model.add(smp.uniform(mu_psi, lower=0 , upper=np.pi))
#     model.add(smp.uniform(mu_theta, lower=0 , upper=np.pi/2))
#     model.add(eta_kappa_function(eta,kappa))
    print(f"Params {params}, Likelihood {likelihood} , Model {model()}")
    return model().round(3)

In [43]:
def main():
    sampler = smp.Metropolis  # Metropolis, Slice, Hamiltonian, NUTS
    num_step = 5 if sampler == smp.NUTS else 50

    Posterior_params = []
    step = 0

    start_params = 0.5, 1, 0.25, 0.2   # List of initial parameters
    scale = {'params': 1}
    while step < num_step:
        print(f'\n______________________________________________{step}th iteration_______________________________________')
        
        start_params_ltk = {'params': start_params}  # Use initial param or updated param
        # print(f'Params start = {start_params_ltk}')  # Print the starting model parameter
        hmc_param_ltk = CustomMetropolis(log_likelihood, start_params_ltk, scale=scale) #sampler(log_likelihood, start_params_ltk) # NUTS, Metropolis, Hamiltonian, Slice
        chain_param_ltk = hmc_param_ltk.sample(1, burn=0, n_chains=1)
        print(f'Accepted params = {chain_param_ltk}')
        # Replace current with accepted params
        start_params = calc_mean(chain_param_ltk.params)
        
        # Store the new params
        Posterior_params.extend(chain_param_ltk.params)
        
        # Repeat step 3-5 
        step += 1
        print(f'Recovered params = {start_params}')

    return Posterior_params
true_param = [ 1 , 1.57 , 0.18, 0.1 ]
print(true_param)
# Simulate the real data
Posterior_params = main()

[1, 1.57, 0.18, 0.1]

______________________________________________0th iteration_______________________________________
Time taken by run_simulation: 46.544410943984985 seconds
Params [0.20592253500428898  1.3864805343563023   0.944451476419681
 0.040893198920281315], Likelihood -29739151.445839852 , Model -29739151.445839852
Time taken by run_simulation: 81.06880211830139 seconds
Params (0.5, 1, 0.25, 0.2), Likelihood -171600718.39645538 , Model -171600718.39645538
Progress: [##############################] 1 of 1 samples
Accepted params = [([0.20592253500428898 , 1.3864805343563023  , 0.944451476419681   , 0.040893198920281315],)]
Recovered params = [0.20592253500428898  1.3864805343563023   0.944451476419681
 0.040893198920281315]

______________________________________________1th iteration_______________________________________
Time taken by run_simulation: 0.9916000366210938 seconds
Params [ 1.079538250646686   1.9176508275421085  2.0166671234012292
 -0.3689477563077036], Likelih

Params [-0.6458104703535938  5.415641527482229   0.7964962334972427
  2.4255124109862085], Likelihood -361228793.63591456 , Model -361228793.63591456
Time taken by run_simulation: 25.67551302909851 seconds
Params [0.6463560207292913  4.853442197777639   0.32610074938620814
 1.2395967310466995 ], Likelihood -4472653.140205219 , Model -4472653.140205219
Progress: [##############################] 1 of 1 samples
Accepted params = [([0.6463560207292913 , 4.853442197777639  , 0.32610074938620814, 1.2395967310466995 ],)]
Recovered params = [0.6463560207292913  4.853442197777639   0.32610074938620814
 1.2395967310466995 ]

______________________________________________22th iteration_______________________________________
Time taken by run_simulation: 0.9250507354736328 seconds
Params [-0.4949181780532286   5.78334787708962     0.12995993218389693
 -0.26243615616933336], Likelihood -558750758.8372719 , Model -558750758.8372719
Time taken by run_simulation: 25.301857948303223 seconds
Params [0.6

Params [-0.4259964359501083  5.152828505965692   1.2387816412839916
  2.9461653597486586], Likelihood -132041256.3622732 , Model -132041256.3622732
Time taken by run_simulation: 30.277422189712524 seconds
Params [0.6463560207292913  4.853442197777639   0.32610074938620814
 1.2395967310466995 ], Likelihood -4587647.377221304 , Model -4587647.377221304
Progress: [##############################] 1 of 1 samples
Accepted params = [([0.6463560207292913 , 4.853442197777639  , 0.32610074938620814, 1.2395967310466995 ],)]
Recovered params = [0.6463560207292913  4.853442197777639   0.32610074938620814
 1.2395967310466995 ]

______________________________________________43th iteration_______________________________________
Time taken by run_simulation: 8.507752895355225 seconds
Params [1.7386555684064169 4.058418073743997  0.6950923241809512
 2.3849502658733286], Likelihood -292768868.78959656 , Model -292768868.78959656
Time taken by run_simulation: 28.38441491127014 seconds
Params [0.6463560207

### Generating 50 microtextures

In [12]:
def background(internal_loop):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, internal_loop, *args, **kwargs)
    return wrapped

class AII_model:
    def __init__(self, eta):
#         eta, kappa = 0.25 , 0.05
        mu_phi, mu_theta, kappa = 1 , 1.57 , 0.15
        self.condRef = 8.7e5
        self.sigxx = 8.97e5
        self.sigzz = 8.43e5
        self.coil_rad_x = 2.1509e6
        self.coil_rad_y = 2.1509e6
        self.mu_phi = mu_phi
        self.mu_theta = mu_theta
        # self.mu_theta2 = mu_theta2
        self.eta = eta
        self.kappa = kappa
        self.data_to_append = []
        self.phi_theta_sigma_data = {}
        self.V1 , self.V2, self.V3 , self.lambda1, self.lambda2, self.lambda3  = self.compute_bingham_distributions()
        self.pdf_value1 = self.calculate_mx_value()
        self.candidates = []
        self.eulerangles = []
        self.microstructure_list = {}
        self.number_of_microstructures = 50
#         self.number_of_microstructures = 1

    async def async_init(self):
        await self.run_simulation()

    def compute_bingham_distributions(self):
        V1 = np.array([
                np.cos(self.mu_theta/2)*np.cos(self.mu_phi/2),
                -1*np.sin(self.mu_theta/2)*np.cos(self.mu_phi/2),
                -1*np.sin(self.mu_theta/2)*np.sin(self.mu_phi/2),
                -1*np.cos(self.mu_theta/2)*np.sin(self.mu_phi/2)
        ])
        V2 = np.array([
            -0.5 * np.cos(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            0.5 * np.sin(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            -0.5 * np.sin(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.cos(self.mu_phi / 2)
        ])
        V3 = np.array([
            -0.5 * np.sin(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.cos(self.mu_phi / 2),
            -0.5 * np.cos(self.mu_theta/2) * np.sin(self.mu_phi / 2),
            0.5 * np.sin(self.mu_theta/2) * np.sin(self.mu_phi / 2)
        ])
        l1 = -0.5
        l2 = l1 / self.eta
        l3 = l2 / self.kappa
        
        return V1 , V2 , V3 , l1 , l2 , l3    #, V21 , V22 , V23

    def pi1_x(self, x):
        term1 = self.lambda1 * (np.dot(self.V1.T, x)) ** 2
        term2 = self.lambda2 * (np.dot(self.V2.T, x)) ** 2
        term3 = self.lambda3 * (np.dot(self.V3.T, x)) ** 2

        sum_terms = term1 + term2 + term3

        pi_x_value = np.exp(sum_terms)
        return pi_x_value
    
    def calculate_mx_value(self):
        return 1
            
    def rununtiltrue(self):
        while True:
            x_candidate = self.generaterandomquaternion()
            f_candidate = self.pi1_x(x_candidate)
            alpha = f_candidate / self.pdf_value1
            if alpha >= 0.98:
                return x_candidate

    def generaterandomquaternion(self):
        # Sample three uniform random variables
        u1, u2, u3 = np.random.rand(3)

        # Compute the quaternion components
        q = np.array([
            np.sqrt(1-u1) * np.sin(2*np.pi*u2),
            np.sqrt(1-u1) * np.cos(2*np.pi*u2),
            np.sqrt(u1) * np.sin(2*np.pi*u3),
            np.sqrt(u1) * np.cos(2*np.pi*u3)
        ])
        return q
    
    @staticmethod
    def quaternion_to_rotation_matrix(q):
        # Compute the associated rotation matrix from the quaternion
        R = np.array([
            [np.square(q[0]) + np.square(q[1]) - np.square(q[2]) - np.square(q[3]), 2*(q[1]*q[2] + q[0]*q[3]), 2*(q[1]*q[3] - q[0]*q[2])],
            [2*(q[1]*q[2] - q[0]*q[3]), np.square(q[0]) - np.square(q[1]) + np.square(q[2]) - np.square(q[3]), 2*(q[3]*q[2] + q[0]*q[1])],
            [2*(q[3]*q[1] + q[0]*q[2]), 2*(q[3]*q[2] - q[0]*q[1]), np.square(q[0]) - np.square(q[1]) - np.square(q[2]) + np.square(q[3])]
        ])
        return R
    
    @staticmethod
    def rotation_matrix_to_euler_angles(R):
        second_angle = np.arccos(R[2][2])
        if second_angle > 0:
            first_angle = np.arctan2(R[2][0], -R[2][1])
            third_angle = np.arctan2(R[0][2], R[1][2])
        elif second_angle == 0:
            first_angle = np.arctan2(R[0][1], R[0][0])
            third_angle = 0
        return first_angle, second_angle, third_angle

    @staticmethod
    def quaternion_angular_difference(q1, q2):
        # Ensure the quaternions are normalized
        q1 = q1 / np.linalg.norm(q1)
        q2 = q2 / np.linalg.norm(q2)
        # Calculate the dot product between the two quaternions
        dot_product = np.dot(q1, q2)
        # Clamp the dot product to avoid numerical issues
        dot_product = np.clip(dot_product, -1.0, 1.0)
        # Calculate the angular difference in radians
        angle = 2 * np.arccos(dot_product)
        # Convert radians to degrees
        return angle
    
    async def process_element(self, i, j, data_list):
        x_candidate1 = self.rununtiltrue()
        self.candidates.append(x_candidate1)
        angle1 = self.rotation_matrix_to_euler_angles(self.quaternion_to_rotation_matrix(x_candidate1))
        first1, second1, third1 = angle1
        self.eulerangles.append(angle1)
        data_list.append((i, j, first1, second1, third1))
            
    async def process_microstructure(self, val):
        """Process each microstructure in parallel."""
        data_list = []
        tasks = []
        for i in range(25):
            for j in range(25):
                tasks.append(self.process_element(i, j, data_list))

        await asyncio.gather(*tasks)
        data_list.sort(key=lambda x: (x[0], x[1]))
        self.microstructure_list[val] = data_list
        # print(f"Printing for {val} microstructure")
    
    async def run_simulation(self):
        """Parallelize the outer loop using asyncio tasks."""
        start_time = time.time()
        outer_tasks = [self.process_microstructure(val) for val in range(1, self.number_of_microstructures+1)]
        await asyncio.gather(*outer_tasks)
        end_time = time.time() - start_time
        print(f"Time taken by run_simulation: {end_time} seconds")

    # def write_data_to_new_file(self,file_path):
    #     # Check if the file exists, delete it if it does
    #     if os.path.exists(file_path):
    #         os.remove(file_path)
    #     # Open the file in append mode ('a') or create it if it doesn't exist ('a+')
    #     with open(file_path, 'a') as file:
    #         # Append each line of data to the file
    #         for line in self.data_to_append[:625]:
    #             file.write(' '.join(map(str, line)) + '\n')  # Add a newline character to separate entries
    
    def write_data_to_new_file(self,file_path):
        # Check if the file exists, delete it if it does
        if os.path.exists(file_path):
            os.remove(file_path)
        # Open the file in append mode ('a') or create it if it doesn't exist ('a+')
        with open(file_path, 'a') as file:
            # Append each line of data to the file
            for line in self.microstructure_list[1]:
                file.write(' '.join(map(str, line)) + '\n')  # Add a newline character to separate entries
                    
                    
    def scan_operation(self): 
        microstructure = self.microstructure_list[1]
        # Extracting X, Y, phi, and theta values from each microstructure
        X = np.array([tup[0] for tup in microstructure]) * (6176/24) + 8
        X = X.reshape((25, 25)).T
        X = np.stack((X, X), axis=2)

        Y = np.array([tup[1] for tup in microstructure]) * (6312/24) + 8
        Y = Y.reshape((25, 25)).T
        Y = np.stack((Y, Y), axis=2)

        self.X = X
        self.Y = Y
            
        for val, microstructure in self.microstructure_list.items():

            phi = np.array([tup[2] for tup in microstructure])
            phi = phi.reshape((25, 25)).T
            phi = np.stack((phi, phi), axis=2)
            cphi = np.cos(phi)
            sphi = np.sin(phi)

            theta = np.array([tup[3] for tup in microstructure])
            theta = theta.reshape((25, 25)).T
            theta = np.stack((theta, theta), axis=2)
            stheta = np.sin(theta)

            # Optionally, you can store these processed arrays in a dictionary if needed
            self.microstructure_list[val] = {
                'cphi': cphi,
                'sphi': sphi,
                'stheta': stheta,
                'microstructure': microstructure
            }
        
        scanDims = {'scanStart': np.array([0.5, 0.5, 0.5]), 'scanLength': np.array([5, 5, 5]), 'scanStep': np.array([0.2, 0.2, 0.2])}

        scanDims['scanStart'] *= 1000
        scanDims['scanLength'] *= 1000
        scanDims['scanStep'] *= 1000

        # Calculate number of points in each dimension
        numXPoints = int(scanDims['scanLength'][0] / scanDims['scanStep'][0]) + 1
        numYPoints = int(scanDims['scanLength'][1] / scanDims['scanStep'][1]) + 1

        # Set up the scan structure
        scan = np.zeros((numXPoints, numYPoints), dtype=object)
        for ixpos in range(numXPoints):
            for iypos in range(numYPoints):
                scan[ixpos, iypos] = {
                    'num': 0,
                    'pos': [
                        scanDims['scanStart'][0] + scanDims['scanStep'][0] * ixpos,
                        scanDims['scanStart'][1] + scanDims['scanStep'][1] * iypos
                    ],
                    'area': [],
                    'total': 0
                }

        Exs = np.load('Exs_25_26_iterations.npy')
        Eys = np.load('Eys_25_26_iterations.npy')

        mul = np.zeros(scan.shape, dtype=np.complex128)

        for val, microstructure_data in self.microstructure_list.items():
            # Initialize scanPoint
            scanPoint = 0

            # Initialize ectScan with the same structure as scan but with all elements set to zero
            ectScan = {
                'val': val,
                'XD': np.zeros(scan.shape, dtype=float),
                'YD': np.zeros(scan.shape, dtype=float),
                'zl': np.zeros(scan.shape, dtype=np.complex128),
                'Xsub_slice': np.empty(scan.shape, dtype=object),  
                'Ysub_slice': np.empty(scan.shape, dtype=object),  
                'Zsub_slice': np.empty(scan.shape, dtype=object)
            }
        
            sphi_sub = microstructure_data['sphi']
            stheta_sub = microstructure_data['stheta']
            cphi_sub = microstructure_data['cphi']

            # Compute sigma values once per microstructure
            sigma_11 = self.sigxx + (self.sigzz - self.sigxx) * sphi_sub ** 2 * stheta_sub ** 2
            sigma_12 = ((self.sigxx - self.sigzz) * (sphi_sub * cphi_sub * (stheta_sub ** 2))) / 2
            sigma_22 = self.sigxx + ((self.sigzz - self.sigxx) * (stheta_sub ** 2)) + ((self.sigxx - self.sigzz) * (sphi_sub ** 2) * (stheta_sub ** 2))

            numX, numY = scan.shape
            real_partf_list = np.empty(scan.shape, dtype=object)
            imag_partf_list = np.empty(scan.shape, dtype=object)
            for ix in range(numX):
                for iy in range(numY):

                    X = self.X
                    Y = self.Y
                    x = np.squeeze(X[0, :, 0])
                    y = np.squeeze(Y[:, 0, 0])

                    # Min and max distances from coil to position in x and y directions
                    cx_lim_low = scan[ix, iy]['pos'][0] - self.coil_rad_x
                    cx_lim_hi = scan[ix, iy]['pos'][0] + self.coil_rad_x
                    cy_lim_low = scan[ix, iy]['pos'][1] - self.coil_rad_y
                    cy_lim_hi = scan[ix, iy]['pos'][1] + self.coil_rad_y

                    # Find the indices for x and y using numpy
                    lowIndX = np.argmax(x >= cx_lim_low)
                    highIndX = np.argmax(x >= cx_lim_hi) - 1 if np.any(x >= cx_lim_hi) else len(x) - 1
                    lowIndY = np.argmax(y >= cy_lim_low)
                    highIndY = np.argmax(y >= cy_lim_hi) - 1 if np.any(y >= cy_lim_hi) else len(y) - 1
                    lowIndZ = 0
                    highIndZ = 1

                    # Extract subarrays for X, Y, Z

                    Xsub = X[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][0]

                    Ysub = Y[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][1]

                    array1 = np.ones((X.shape[0], X.shape[1], 1)) * 8
                    array2 = np.ones((X.shape[0], X.shape[1], 1)) * 72

                    # Stack these arrays along the third dimension
                    Zsub = np.concatenate((array1, array2), axis=2)
                    Ex = Exs[scanPoint]   
                    Ey = Eys[scanPoint]

                    # Components of the approximate impedance integral (AII)
                    f1 = Ex ** 2 * (self.condRef - sigma_11)
                    f2 = -2 * Ex * Ey * sigma_12
                    f3 = Ey ** 2 * (self.condRef - sigma_22)
                    f = f1 + f2 + f3
                    real_partf_list[ix, iy] = np.real(f)
                    imag_partf_list[ix, iy] = np.imag(f)
                    
                    Xsub_slice = (np.squeeze(Xsub[0, :, 0]) / 1e6)
                    Ysub_slice = (np.squeeze(Ysub[:, 0, 0]) / 1e6)
                    Zsub_slice = (np.squeeze(Zsub[0, 0, :]) / 1e6)
                    
                    integral_inner = np.trapz(f, x=Xsub_slice, axis=1).reshape(25,1,2)
                    integral_middle = np.trapz(integral_inner, x=Ysub_slice, axis=0).reshape(1,1,2)
                    zl = np.trapz(integral_middle, x=Zsub_slice, axis=2)

                    ectScan['XD'][ix, iy] = scan[ix, iy]['pos'][0] / 1000  # Back to mm
                    ectScan['YD'][ix, iy] = scan[ix, iy]['pos'][1] / 1000  # Back to mm
                    ectScan['Xsub_slice'][ix, iy] = Xsub_slice
                    ectScan['Ysub_slice'][ix, iy] = Ysub_slice
                    ectScan['Zsub_slice'][ix, iy] = Zsub_slice
                    ectScan['zl'][ix, iy] = zl
                    
                    mul[ix, iy] += ectScan['zl'][ix, iy]
                    
                    scanPoint += 1
            microstructure_data['ectScan'] = ectScan
            microstructure_data['realpartlist'] = real_partf_list
            microstructure_data['imagpartlist'] = imag_partf_list

        mul = mul/self.number_of_microstructures

        # Store the 'mul' result back in self if needed
        self.mul = mul

    def calculate_covariance_matrix(self):
        num_microstructures = len(self.microstructure_list)
        num_elements = 676  # 26 * 26
        single_side = 26
        # Initialize covariance matrices
        gamma = np.zeros((num_elements, num_elements), dtype=np.complex128)
        gamma_hat = np.zeros((num_elements, num_elements), dtype=float)
        
        for l in range(num_elements):
            for k in range(l + 1):
                sum_zl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    ectScan = microstructure_data['ectScan']
                    zl_flatten = ectScan['zl'].flatten()
                    # Get zl for l and k
                    zl_l = zl_flatten[l]
                    zl_k = zl_flatten[k]

                    sum_zl_product += zl_l * zl_k 
                    
                gamma[l, k] = sum_zl_product / self.number_of_microstructures  - (self.mul.flat[l]*self.mul.flat[k])
                gamma[k, l] = gamma[l, k]
            
        self.gamma = gamma
        
        for l in range(num_elements):  # num_elements is the size of the matrix
            for k in range(l + 1):
                sum_zlk_product = 0
                sum_zkl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    i, j = divmod(l, single_side)
                    m, n = divmod(k, single_side)

                    realpartl = microstructure_data['realpartlist'][i,j]
                    imagpartl = microstructure_data['imagpartlist'][i,j]
                    
                    realpartk = microstructure_data['realpartlist'][m,n]
                    imagpartk = microstructure_data['imagpartlist'][m,n]
                    
                    Xsub_slicel = microstructure_data['ectScan']['Xsub_slice'][i,j]
                    Ysub_slicel = microstructure_data['ectScan']['Ysub_slice'][i,j]
                    Zsub_slicel = microstructure_data['ectScan']['Zsub_slice'][i,j]
                    
                    Xsub_slicek = microstructure_data['ectScan']['Xsub_slice'][m,n]
                    Ysub_slicek = microstructure_data['ectScan']['Ysub_slice'][m,n]
                    Zsub_slicek = microstructure_data['ectScan']['Zsub_slice'][m,n]
                    
                    integral_inner_l_lk = np.trapz(realpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_lk = np.trapz(integral_inner_l_lk, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zlk_l = np.trapz(integral_middle_l_lk, x=Zsub_slicel, axis=2)

                    integral_inner_k_lk = np.trapz(imagpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_lk = np.trapz(integral_inner_k_lk, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zlk_k = np.trapz(integral_middle_k_lk, x=Zsub_slicek, axis=2)

                    integral_inner_k_kl = np.trapz(realpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_kl = np.trapz(integral_inner_k_kl, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zkl_k = np.trapz(integral_middle_k_kl, x=Zsub_slicek, axis=2)

                    integral_inner_l_kl = np.trapz(imagpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_kl = np.trapz(integral_inner_l_kl, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zkl_l = np.trapz(integral_middle_l_kl, x=Zsub_slicel, axis=2)
                    
                    # Compute product and accumulate
                    sum_zlk_product += zlk_l * zlk_k
                    sum_zkl_product += zkl_l * zkl_k

                # Calculate the average across all microstructures
                gamma_hat[l, k] = sum_zlk_product / self.number_of_microstructures  - (np.real(self.mul.flat[l])*np.imag(self.mul.flat[k]))
                gamma_hat[k, l] = sum_zkl_product / self.number_of_microstructures  - (np.imag(self.mul.flat[l])*np.real(self.mul.flat[k]))

        self.gamma_hat = gamma_hat

In [15]:
aiimodel=AII_model(0.1)
await aiimodel.async_init()
aiimodel.scan_operation()

Time taken by run_simulation: 401.793673992157 seconds


In [16]:
z_re = np.real(aiimodel.microstructure_list[1]['ectScan']['zl']).flatten()
z_im = np.imag(aiimodel.microstructure_list[1]['ectScan']['zl']).flatten()

In [17]:
# For eta 0.5
likelihhod_values = []
for val in np.arange(-0.5, 0.6, 0.1):
    likelihhod_values.append(log_likelihood(val))

Time taken by run_simulation: 1.039052963256836 seconds
Params -0.5, Likelihood -665192524.8744357 , Model -665192524.8744357
Time taken by run_simulation: 1.1183149814605713 seconds
Params -0.4, Likelihood -642764413.6940128 , Model -642764413.6940128
Time taken by run_simulation: 1.0458543300628662 seconds
Params -0.30000000000000004, Likelihood -657502553.9984354 , Model -657502553.9984354
Time taken by run_simulation: 1.0146422386169434 seconds
Params -0.20000000000000007, Likelihood -649351418.3056356 , Model -649351418.3056356
Time taken by run_simulation: 1.0681841373443604 seconds
Params -0.10000000000000009, Likelihood -644038026.1181455 , Model -644038026.1181455


/var/folders/fd/lndhxkqs20bb__svt4c6nddr0000gn/T/ipykernel_21956/4053365115.py:65: RuntimeWarning: overflow encountered in exp
  pi_x_value = np.exp(sum_terms)


Time taken by run_simulation: 1.2026729583740234 seconds
Params -1.1102230246251565e-16, Likelihood -607928200.6370629 , Model -607928200.6370629
Time taken by run_simulation: 427.4268448352814 seconds
Params 0.09999999999999987, Likelihood -1533299.8836508198 , Model -1533299.8836508198
Time taken by run_simulation: 199.69959592819214 seconds
Params 0.19999999999999984, Likelihood -2033938.687894111 , Model -2033938.687894111
Time taken by run_simulation: 128.87726998329163 seconds
Params 0.2999999999999998, Likelihood -3059502.6616325313 , Model -3059502.6616325313
Time taken by run_simulation: 103.91634631156921 seconds
Params 0.3999999999999998, Likelihood -4744195.853617466 , Model -4744195.853617466
Time taken by run_simulation: 78.54592990875244 seconds
Params 0.4999999999999998, Likelihood -7160220.159180797 , Model -7160220.159180797


In [ ]:
def main():
    sampler = smp.Metropolis  # Metropolis, Slice, Hamiltonian, NUTS
    num_step = 5 if sampler == smp.NUTS else 50

    Posterior_params = []
    step = 0

    start_params = 0.5  # List of initial parameters
    scale = {'params': 1}
    while step < num_step:
        print(f'\n______________________________________________{step}th iteration_______________________________________')
        
        start_params_ltk = {'params': start_params}  # Use initial param or updated param
        # print(f'Params start = {start_params_ltk}')  # Print the starting model parameter
        hmc_param_ltk = CustomMetropolis(log_likelihood, start_params_ltk, scale=scale) #sampler(log_likelihood, start_params_ltk) # NUTS, Metropolis, Hamiltonian, Slice
        chain_param_ltk = hmc_param_ltk.sample(1, burn=0, n_chains=1)
        print(f'Accepted params = {chain_param_ltk}')
        # Replace current with accepted params
        start_params = calc_mean(chain_param_ltk.params)
        
        # Store the new params
        Posterior_params.extend(chain_param_ltk.params)
        
        # Repeat step 3-5 
        step += 1
        print(f'Recovered params = {start_params}')

    return Posterior_params
start_param = [ 1 , 1.57 , 0.5 , 0.15 ]
print(start_param)
# Simulate the real data
Posterior_params = main()